In [1]:
import pandas as pd
import cv2
import seaborn as sns
import numpy as np
from tqdm.notebook import tqdm
from multiprocessing import Pool
import os
import copy
from datetime import datetime
from sklearn.model_selection import train_test_split

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.applications.vgg16 import VGG16
from keras.applications.resnet_v2 import ResNet152V2, ResNet50V2
from keras.applications.densenet import DenseNet121, DenseNet169
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.nasnet import NASNetMobile
from keras.preprocessing.image import ImageDataGenerator
from keras.backend import clear_session

import tensorflow as tf

Using TensorFlow backend.


In [2]:
tqdm.pandas()

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
dataFolder="./data/input"
imageFolder="./data/input/train_images"
evalImageFolder="./data/input/test_images"
size=256

In [5]:
df=pd.read_csv("./data/input/train_labels.csv", header=None)
df=df.set_index(0)

In [6]:
df["classifier"]=df[1]-1979
df["regressor"]=df["classifier"]/(2018-1979)
df.drop([1],axis=1,inplace=True)

In [7]:
def getImage(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(size,size))
    img = np.real(img)
    img = img.astype(np.float32)
    return img

In [8]:
with Pool() as p:
    imap=p.imap(getImage,sorted([os.path.join(imageFolder,i) for i in os.listdir(imageFolder)]))
    imgs=list(tqdm(imap,total=len(os.listdir(imageFolder))))
imgs=np.asarray(imgs)/255

with Pool() as p:
    imap=p.imap(getImage,sorted([os.path.join(evalImageFolder,i) for i in os.listdir(evalImageFolder)]))
    evalImgs=list(tqdm(imap,total=len(os.listdir(evalImageFolder))))
evalImgs=np.asarray(evalImgs)/255

In [9]:
def getRes152BasedModel():
    model = ResNet152V2(weights=None, include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation="relu")(x)
    classifier = Dense(40, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=classifier)
    return model

def getDense121BasedModel():
    model = DenseNet121(weights=None, include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation="relu")(x)
    classifier = Dense(40, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=classifier)
    return model

def getInceptionResBasedModel():
    model = InceptionResNetV2(weights=None, include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation="relu")(x)
    classifier = Dense(40, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=classifier)
    return model

def getRes50BasedModel():
    model = ResNet50V2(weights=None, include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation="relu")(x)
    classifier = Dense(40, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=classifier)
    return model

def getDense169BasedModel():
    model = DenseNet169(weights=None, include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation="relu")(x)
    classifier = Dense(40, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=classifier)
    return model

In [10]:
#X_train, X_test, X_train_fft, X_test_fft, y_train, y_test = train_test_split(imgs, fftimgs, df,train_size=0.9,random_state=8000)
X_train, X_test, y_train, y_test = train_test_split(imgs, df,train_size=0.9,random_state=8000)

In [11]:
def loss_fn(y_true, y_pred):
    num_classes = 40
    # クロスエントロピー
    ce = tf.keras.losses.categorical_crossentropy

    loss = 0

    # 正解との誤差
    loss += 0.98 * ce(y_true, y_pred)

    # "正解 - 1"との誤差
    mask = np.array([1] * (num_classes - 1) + [0], dtype="float32")
    loss += 0.01 * ce(mask * tf.roll(y_true, shift=-1, axis=1), y_pred)

    # "正解 + 1"との誤差
    mask = np.array([0] + [1] * (num_classes - 1), dtype="float32")
    loss += 0.01 * ce(mask * tf.roll(y_true, shift=1, axis=1), y_pred)
    return loss


def score_fn(y_true, y_pred):
    y_true = tf.math.argmax(y_true, axis=-1, output_type="int32")
    y_pred = tf.math.argmax(y_pred, axis=-1, output_type="int32")
    return tf.cast(tf.less_equal(tf.math.abs(y_true - y_pred), 1), "float32")

In [12]:
early_stopping =  EarlyStopping(monitor='val_loss', min_delta=0.0, patience=8)

In [ ]:
for i in tqdm(range(100)):
    for modelfunc in [getRes152BasedModel, getDense121BasedModel,getInceptionResBasedModel,getRes50BasedModel,getDense169BasedModel]:
        model = modelfunc()
        print(modelfunc)
        model.compile(optimizer=Adam(), metrics=["acc",score_fn], loss=loss_fn)
        model.fit(X_train, to_categorical(y_train["classifier"]), epochs=80, validation_data=(X_test,to_categorical(y_test["classifier"])), callbacks=[early_stopping])
        os.makedirs("./models/normal",exist_ok=True)
        date= int(float(datetime.now().timestamp()))
        model.save('./models/normal/{0}.h5'.format(date))
        y_pred=model.predict(evalImgs)
        date= int(float(datetime.now().timestamp()))
        np.save("./result/classifier/{0}.npy".format(date), y_pred)
        del model
        clear_session()

<function getRes152BasedModel at 0x7efd2f7bc400>
Train on 6017 samples, validate on 669 samples
Epoch 1/80
6017/6017 [==============================] - 245s 41ms/step - loss: 3.3377 - acc: 0.0986 - score_fn: 0.1949 - val_loss: 3.6701 - val_acc: 0.0658 - val_score_fn: 0.1330
Epoch 2/80
6017/6017 [==============================] - 137s 23ms/step - loss: 3.2276 - acc: 0.1122 - score_fn: 0.2144 - val_loss: 12.2744 - val_acc: 0.0613 - val_score_fn: 0.1315
Epoch 3/80
6017/6017 [==============================] - 137s 23ms/step - loss: 3.1145 - acc: 0.1293 - score_fn: 0.2431 - val_loss: 3.6032 - val_acc: 0.1181 - val_score_fn: 0.2152
Epoch 4/80
6017/6017 [==============================] - 137s 23ms/step - loss: 3.0288 - acc: 0.1409 - score_fn: 0.2626 - val_loss: 7.9809 - val_acc: 0.0478 - val_score_fn: 0.0822
Epoch 5/80
6017/6017 [==============================] - 138s 23ms/step - loss: 2.9777 - acc: 0.1482 - score_fn: 0.2711 - val_loss: 3.4074 - val_acc: 0.1001 - val_score_fn: 0.2093
Epoch 6/

2400/6017 [==========>...................] - ETA: 1:19 - loss: 1.4070 - acc: 0.5663 - score_fn: 0.7138

In [ ]:
print(modelfunc)